In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd


In [4]:
data_train = pd.read_json("train.jsonl", lines=True)
data_test = pd.read_json("test.jsonl", lines=True)
data_dev = pd.read_json("dev.jsonl", lines=True)

print(len(data_train), "Training")
print(len(data_test), "Test")
print(len(data_dev), "Dev")

combined_data = pd.concat([data_train, data_dev], ignore_index=True)
combined_data = data_train
#combined_data.head()

print(len(combined_data), "combined")

8739 Training
803 Test
343 Dev
8739 combined


In [6]:
#resample data, oversampling
import pandas as pd
from sklearn.utils import resample

def oversample_and_shuffle(df):
    # Separate classes
    df_class_minus_one = df[df['polarity'] == -1]
    df_class_0 = df[df['polarity'] == 0]
    df_class_1 = df[df['polarity'] == 1]

    # Oversample minority classes
    df_class_minus_one_oversampled = resample(df_class_minus_one, replace=True, n_samples=len(df_class_0), random_state=42)
    df_class_1_oversampled = resample(df_class_1, replace=True, n_samples=len(df_class_0), random_state=42)

    # Combine oversampled minority classes with majority class
    df_oversampled = pd.concat([df_class_minus_one_oversampled, df_class_0, df_class_1_oversampled])

    df_oversampled = df_oversampled.sample(frac=1, random_state=42)

    # Display the counts after oversampling
    print(df_oversampled['polarity'].value_counts())

    return df_oversampled

df = combined_data
combined_data = oversample_and_shuffle(df)

df = data_test
data_test = oversample_and_shuffle(df)

polarity
 1    3028
 0    3028
-1    3028
Name: count, dtype: int64
polarity
 1    319
 0    319
-1    319
Name: count, dtype: int64


In [7]:
combined_data.head()
# Specify the columns to be removed
columns_to_remove = ['mention', 'from', 'to', 'id']

# Use the drop method to remove the specified columns
combined_data = combined_data.drop(columns=columns_to_remove)
data_test = data_test.drop(columns=columns_to_remove)



In [8]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

embedding_dim = 100
vocab_size = 18000
max_length = 200

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embedding_dim)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)

num_classes = 3
outputs = layers.Dense(num_classes, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1800000   
                                                                 
 bidirectional (Bidirection  (None, None, 128)         84480     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 1983683 (7.57 MB)
Trainable params: 1983683 (7

In [9]:
combined_data = combined_data.drop_duplicates(subset='sentence', keep='first')
data_test = data_test.drop_duplicates(subset='sentence', keep='first')

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

texts_train = combined_data['sentence']
labels_train = combined_data['polarity']

texts_test = data_test['sentence']
labels_test = data_test['polarity']

# Tokenize the text data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts_train)
sequences_train = tokenizer.texts_to_sequences(texts_train)

tokenizer_test = Tokenizer(num_words=vocab_size)
tokenizer_test.fit_on_texts(texts_test)
sequences_test = tokenizer_test.texts_to_sequences(texts_test)

maxlen = 200
x_train = pad_sequences(sequences_train, maxlen=maxlen)
x_val = pad_sequences(sequences_test, maxlen=maxlen)

y_train = labels_train
y_val = labels_test

In [11]:
num_classes = len(np.unique(y_train))
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_val_categorical = to_categorical(y_val, num_classes=num_classes)

In [12]:
print(len(combined_data), "combined")
print(len(data_test), "test")
print(len(tokenizer.word_index), "word index")

5392 combined
539 test
16011 word index


In [13]:
model.fit(x_train, y_train_categorical, epochs=10, batch_size=64, validation_data=(x_val, y_val_categorical), callbacks=[early_stopping])


loss, accuracy = model.evaluate(x_val, y_val_categorical)
print(f'Loss: {loss}, Accuracy: {accuracy * 100:.2f}%')


Epoch 1/10


85/85 [==============================] - 17s 151ms/step - loss: 1.0384 - accuracy: 0.5167 - val_loss: 0.9879 - val_accuracy: 0.5529
Epoch 2/10
85/85 [==============================] - 11s 134ms/step - loss: 1.0009 - accuracy: 0.5347 - val_loss: 0.9826 - val_accuracy: 0.5529
Epoch 3/10
85/85 [==============================] - 11s 133ms/step - loss: 0.9877 - accuracy: 0.5347 - val_loss: 0.9803 - val_accuracy: 0.5529
Epoch 4/10
85/85 [==============================] - 12s 136ms/step - loss: 0.9468 - accuracy: 0.5475 - val_loss: 0.9954 - val_accuracy: 0.5306
Epoch 5/10
85/85 [==============================] - 11s 134ms/step - loss: 0.7934 - accuracy: 0.6434 - val_loss: 1.1269 - val_accuracy: 0.4879
Epoch 6/10
17/17 [==============================] - 0s 26ms/step - loss: 1.3385 - accuracy: 0.4731
Loss: 1.3385272026062012, Accuracy: 47.31%


In [15]:
from sklearn.metrics import precision_score, recall_score


# Predict on validation data
y_val_pred_prob = model.predict(x_val)
y_val_pred = np.argmax(y_val_pred_prob, axis=1)  # Convert probabilities to class labels

y_val_true = np.argmax(y_val_categorical, axis=1)

# Calculate precision and recall for each class
precision = precision_score(y_val_true, y_val_pred, average='macro') 
recall = recall_score(y_val_true, y_val_pred, average='macro')  

# Display the results
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')



17/17 [==============================] - 0s 24ms/step
Precision: 0.3392
Recall: 0.3451
